In [1]:
import numpy as np
import pandas as pd
from datetime import date

In [2]:
RAW = pd.read_csv(r'C:\Users\User\VSCode\CDSS\source\Clean\CDSS_Row_1110609.csv', dtype={'病歷號碼':'str'}, sep=',')

C:\Users\User\AppData\Local\Temp\ipykernel_11816\3600871067.py:1: DtypeWarning: Columns (9,14,19,32,38,42,43,45,46,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  RAW = pd.read_csv(r'C:\Users\User\VSCode\CDSS\source\Clean\CDSS_Row_1110609.csv', dtype={'病歷號碼':'str'}, sep=',')


In [3]:
RAW = RAW.drop_duplicates()

In [4]:
RAW.columns

Index(['表單號', '住院號', '申請編號', '病歷號碼', '生日(年月日)', '主診斷1', '診斷2', '診斷3', '診斷4',
       '診斷5', '開單醫師', '開單科別', '病房', '住院日', '出院日', '申請日', '採檢日', '簽收日', '報告日',
       '項目代碼', '項目名稱', '來源', '檢體', '菌名', 'strain', 'bed', '檢驗部收到時間', '發報告時間',
       '參考值', 'Lab_檢驗日期', 'Lab_報告日期', 'Lab_檢驗項目', 'Lab_檢驗值', 'Lab_表單號',
       'Lab_申請號', '床號', '性別', '体重', '死亡日期', '時間', 'ICU', '就醫序號', 'openTime',
       '菌株', '預測值', '說明', 'sendTime', 'fileName', 'miniTime_Open', 'PredLevel',
       'Group_Germ'],
      dtype='object')

In [5]:
print(RAW[['住院號', '病歷號碼']].drop_duplicates().count())
print()
RAW[RAW['菌名'].str.contains('MRSA')][['住院號', '病歷號碼']].drop_duplicates().count()

住院號     8238
病歷號碼    8238
dtype: int64



住院號     733
病歷號碼    733
dtype: int64

In [4]:
RAW = RAW[RAW['菌株'].notna()]
RAW_SA = RAW[RAW['菌株'].str.contains('Staphylococcus aureus')].drop_duplicates()
RAW_SA = RAW_SA[['住院號', '病歷號碼', '菌株', '採檢日', '報告日', 'sendTime']].drop_duplicates()
RAW_SA.採檢日 = pd.to_datetime(RAW_SA.採檢日)
RAW_SA.報告日 = pd.to_datetime(RAW_SA.報告日)
RAW_SA.sendTime = pd.to_datetime(RAW_SA.sendTime)

In [6]:
SMS = pd.read_excel(r'C:\Users\User\VSCode\CDSS\source\簡訊通知時間(0301-0531)給鈞威.xls', dtype={'病歷號_10碼':'str'})

# SMS = SMS[['病歷號_10碼', '西元採檢日', '簡訊通知日期_西元', '護理站', '床號', '檢體', '菌株', '預測值(%)', '實際報告']]
SMS.rename(columns = {'病歷號_10碼':'病歷號碼'}, inplace = True)
SMS.西元採檢日 = pd.to_datetime(SMS.西元採檢日.astype('str'))
SMS.簡訊通知日期_西元 = pd.to_datetime(SMS.簡訊通知日期_西元.astype('str'))
SMS_SA = SMS[SMS['菌株'].str.contains('MRSA')].drop_duplicates()

In [6]:
SMS_list = pd.merge(RAW_SA, SMS_SA[['病歷號碼', '西元採檢日', '簡訊通知日期_西元']], how='left', on='病歷號碼').drop_duplicates()
SMS_list = SMS_list[(SMS_list.西元採檢日.notna())]
SMS_list.sendTime = SMS_list.sendTime.dt.date

SMS_list = SMS_list[(SMS_list.簡訊通知日期_西元 >= SMS_list.採檢日) & (SMS_list.簡訊通知日期_西元 <= SMS_list.報告日) & (SMS_list.簡訊通知日期_西元 >= SMS_list.sendTime)].drop_duplicates()
SMS_list['SMS'] = 1

In [7]:
SMS_list = SMS_list[['住院號', '病歷號碼', 'SMS']].drop_duplicates()

In [8]:
SA = pd.read_excel(r'C:\Users\User\VSCode\CDSS\source\Clean\SA_TableOne_Data_0623(MRSA、MSSA).xlsx', dtype={'病歷號碼':'str'})


In [9]:
SA_SMS = pd.merge(SA, SMS_list, how='left', on=['住院號', '病歷號碼'])
SA_SMS.SMS = np.where(SA_SMS.SMS==1, 1, 0)

In [11]:
SA_SMS.to_excel(r'C:\Users\User\VSCode\CDSS\output\SA_TableOne_Data_0623_SMS(MRSA、MSSA).xlsx', index=False)